# DNA Center SDK

In [ ]:
from dnac_config import DNAC, DNAC_PORT
import json

# this is only requiired for proxy change
import requests
requests.packages.urllib3.disable_warnings()
print(DNAC)

In [ ]:
from dnacentersdk import api

In [ ]:
# only need to verify=False as going to change thte base_url later due to proxy
dnac = api.DNACenterAPI(username="devnetuser",
                        password="Cisco123!",
                        base_url="https://{}:443".format(DNAC),verify=False)
print(dnac.base_url)

In [ ]:
devices = dnac.devices.get_device_list()

#### By default the object that is returned can be rendered as a json.dict

In [ ]:
print(json.dumps(devices,indent=2))

In [ ]:
# old way.. json attributes
data = ", ".join([ "{}:{}".format(dev['managementIpAddress'],dev['platformId'])
                  for dev in devices['response']])

#### Example of an object

In [ ]:
# new way... objects  dev.managementIpAddress vs dev['managementIpAddress']
data = ", ".join([ "{}:{}".format(dev.managementIpAddress,dev.platformId)
                  for dev in devices.response])

In [ ]:
print(data)

## Templates

In [ ]:
templates = dnac.template_programmer.gets_the_templates_available()

In [ ]:
template = [ t for t in templates if t.projectName == "switching" and t.name == "interface-des"][0]

In [ ]:
print(json.dumps(template, indent=2))

In [ ]:
# this is the ID of the template object.
templateId= template['templateId']


In [ ]:
print(templateId)

In [ ]:
# this will find the latest version of the template (inside the object).  NOTE The ID are different..
latest_id = 0
max = 0
for v in template['versionsInfo']:
    if int(v['version']) > max:
        max = int(v['version'])
        latest_id = v['id']
print(latest_id)

In [ ]:
# the master template contains the vars required, and the body of the template.
templateData = dnac.template_programmer.get_template_details(template_id=templateId)


In [ ]:
print(json.dumps(templateData, indent=2))

In [ ]:
payload = {
    "templateId": latest_id,
    "forcePushTemplate" : True,
    "targetInfo": [
     {

        "id": "10.10.20.81",
        "type": "MANAGED_DEVICE_IP",
        "params": {"description": "my Desc", "interface":"g1/0/12"}
        }
     ]
    }

In [ ]:
# this is a hack for the proxy in sandbox.  means we can use RW API with RO creds.  #primativeRBAC
dnac.base_url="https://sandboxdnac2.cisco.com:8080"


In [ ]:
deployment= dnac.template_programmer.deploy_template(payload=payload)

In [ ]:
print (json.dumps(deployment,indent=2))

In [ ]:
deploymentId = deployment.deploymentId.split(":")[-1].strip()
print(deploymentId)

In [ ]:
status = dnac.template_programmer.get_template_deployment_status(deploymentId)

In [ ]:
print(json.dumps(status,indent=2))